## Create FHIRSTU3 SearchParameter Resource, CababilityStatement and Quick start text, and Searchparameter list. Using the Python FHIRClient Module and SearchParameter CSV file with source data.

### Prerequisites:

- Python 3.6 or greater


### Import FHIRClient and other libraries

In [368]:
%config IPCompleter.greedy=True

In [401]:
from fhirclient.models import searchparameter as SP
from fhirclient.models import capabilitystatement as CS
import fhirclient.models.identifier as I
import fhirclient.models.coding as C
import fhirclient.models.codeableconcept as CC
import fhirclient.models.fhirdate as D
import fhirclient.models.extension as X
import fhirclient.models.contactdetail as CD
from json import dumps
from requests import get, post, put
import os
from csv import reader as csvreader
from IPython.display import display, Markdown, HTML
from pprint import pprint
from collections import namedtuple
from pandas import read_csv
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape

#### Inspect ElementProperties as reference

####  Assign Global Variables


Here is where we assign all the global variables for this example such as the local paths for file input and output

In [418]:
in_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
out_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
#out_path = "C:/Users/Eric/Documents/Jan_2019_FHIR_Experience"
sp_csv = 'SearchParams.csv'
md_template = ['search_narrative.j2', 'sp_list_page.j2', 'cs_search_documentation.j2']

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

fhir_term_server = 'http://test.fhir.org/r3'
fhir_test_server = 'http://test.fhir.org/r3'

headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json'
}

# profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
fhir_base_url = 'http://hl7.org/fhir/'
base_id = "argo-q"
canon_base = "canon_base-for-argo-q/"
publisher = 'The Argonaut Project'
publisher_endpoint = dict(
                        system = 'url',
                        value = 'https://github.com/argonautproject/questionnaire/issues'
                        )
none_list = ['', ' ', 'none', 'n/a', 'N/A']
sep_list = (',', ';', ' ', ', ', '; ')
search_type = dict(
    token = '{[system]}|[code]',
    id = '[id]',
    reference = '[url]',
    string = '[string]'
    )


### Get Search Parameter input data

In [313]:
data = read_csv(f'{in_path}{sp_csv}')
data

,Resource,Conformance Server,Parameter,Exists,Type,Expression,Modifiers,Comparators,Reference,Chains,Description,Example Query,foo
0,HealthcareService,SHALL,active,Y,token,HealthcareService.active,NaN,NaN,NaN,NaN,The Healthcare Service is currently marked as ...,GET [base]/HealthcareService?active=[code],NaN
1,Organization,SHALL,active,Y,token,Organization.active,NaN,NaN,NaN,NaN,Is the Organization record active,GET [base]/Organization?active=[code],NaN
2,OrganizationAffiliation,SHALL,active,Y,token,OrganizationAffiliation.active,NaN,NaN,NaN,NaN,Select roles that are active/inactive,GET [base]/OrganizationAffiliation?active=[code],NaN
3,Practitioner,SHALL,active,Y,token,Practitioner.active,NaN,NaN,NaN,NaN,Whether the practitioner record is active,GET [base]/Practitioner?active=[code],GET [Base]/Practitioner?family=[string]&=given...
4,PractitionerRole,SHALL,active,Y,token,PractitionerRole.active,NaN,NaN,NaN,NaN,Whether this practitioner role record is in ac...,GET [base]/PractitionerRole?active=[code],NaN
5,Organization,MAY,address-use,Y,token,Organization.address.use,NaN,NaN,NaN,NaN,Select organizations with an address for the s...,GET [base]/Organization?address-use=[code],GET [base]/Practitioner?qualification-issuer=[...
6,CareTeam,MAY,careteam-category,Y,token,CareTeam.category,text,NaN,NaN,NaN,Select CareTeams of the specified type,GET [base]/CareTeam?careteam-category=[code],GET [base]/Practitioner?identifier-assigner=[url]
7,CareTeam,SHOULD,careteam-endpoint,N,reference,CareTeam.extension('http://hl7.org/fhir/ig/vhd...,NaN,NaN,Endpoint,identifier; connection-type; organization,Select CareTeams with the specified endpoint,GET [base]/CareTeam?careteam-endpoint=[id],NaN
8,CareTeam,SHALL,careteam-identifier,Y,token,CareTeam.identifier,text; ofType,NaN,NaN,NaN,Select CareTeams with the specified identifier,GET [base]/CareTeam?careteam-identifier=[code],NaN
9,CareTeam,MAY,careteam-identifier-assigner,N,reference,CareTeam.identifier.assigner,below,NaN,Organization,identifier; name,Select care teams with an identifier assigned ...,GET [base]/CareTeam?careteam-identifier-assign...,NaN


In [314]:
with open(f'{in_path}{sp_csv}') as f:
    reader = csvreader(f)
    top_row = next(reader)
    top_row = [t.lower().split(' ') for t in top_row]
    top_row = ['_'.join(t) for t in top_row]

    Data = namedtuple("Data", top_row)
    data = [Data(*r) for r in reader]
    
for d in data:
    print(f'Resource = {d.resource}, Search Parameter = {d.parameter}')
    

Resource = HealthcareService, Search Parameter = active
Resource = Organization, Search Parameter = active
Resource = OrganizationAffiliation, Search Parameter = active
Resource = Practitioner, Search Parameter = active
Resource = PractitionerRole, Search Parameter = active
Resource = Organization, Search Parameter = address-use
Resource = CareTeam, Search Parameter = careteam-category
Resource = CareTeam, Search Parameter = careteam-endpoint
Resource = CareTeam, Search Parameter = careteam-identifier
Resource = CareTeam, Search Parameter = careteam-identifier-assigner
Resource = CareTeam, Search Parameter = careteam-location
Resource = CareTeam, Search Parameter = careteam-member
Resource = CareTeam, Search Parameter = careteam-name
Resource = CareTeam, Search Parameter = careteam-organization
Resource = CareTeam, Search Parameter = careteam-service
Resource = CareTeam, Search Parameter = careteam-status
Resource = CareTeam, Search Parameter = careteam-via-intermediary
Resource = Endp

### Create SP Resources by assigning CSV data to SP elements

In [315]:
sp_list = []



def split_string(s):
    for r in sep_list:
        s = s.replace(r, ' ')
    return s.split()

for d in data:
    #initiliaze
    sp = SP.SearchParameter()
    sp.date = D.FHIRDate(datetime.utcnow().isoformat())
    sp.publisher = publisher
    sp.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
    # print(d.parameter)
    sp.id = f'{base_id}-{d.resource.lower()}-{d.parameter}'
    sp.url = f'{canon_base}{sp.id}'
    sp.name = sp.id.replace('-','_')
    sp.derivedFrom = f'http://hl7.org/fhir/SearchParameter/{d.resource}-{d.parameter.lower()}' if d.exists == 'Y' else None
    # print(sp.derivedFrom)
    sp.status = 'active'
    sp.description = d.description
    sp.code = d.parameter.lower()
    sp.base = [d.resource]  # list
    sp.type = d.type.lower()
    # print(sp.type)
    sp.expression = d.expression
    sp.target = [] if d.reference in none_list else split_string(d.reference) # string as space separated list in csv
    # print(sp.target)
    # sp.multipleOr = True  add columns to csv for where needed or used otherwise leave blank
    # sp.multipleAnd = True  add columns to csv for where needed or used
    sp.comparator = [] if d.comparators in none_list else split_string(d.comparators) # string as space separated list in csv
    # print(sp.comparator)
    sp.modifier = [] if d.modifiers in none_list else split_string(d.modifiers) # string as space separated list in csv
    # print(sp.modifier)
    sp.chain = [] if d.chains in none_list else split_string(d.chains) # string as space separated list in csv
    # print(sp.chain)
    # TODO sp.combo = [] if d.combos in none_list else split_string(d.chains) # string as space separated list in csv
    # print(sp.combo)
    sp.conformance = d.conformance_server.upper()
    
    sp_list.append(sp)
    
    


### Display SP resources

In [295]:
for row,i in enumerate(sp_list):
    print(row,i.base)
    print(dumps(sp_list[7].as_json(), indent = 3))



0 ['HealthcareService']
{
   "id": "argo-q-careteam-careteam-endpoint",
   "base": [
      "CareTeam"
   ],
   "chain": [
      "identifier",
      "connection-type",
      "organization"
   ],
   "code": "careteam-endpoint",
   "contact": [
      {
         "telecom": [
            {
               "system": "url",
               "value": "https://github.com/argonautproject/questionnaire/issues"
            }
         ]
      }
   ],
   "date": "2019-01-29T04:32:28.828666",
   "description": "Select CareTeams with the specified endpoint",
   "expression": "CareTeam.extension('http://hl7.org/fhir/ig/vhdir/StructureDefinition/endpoint-reference')",
   "name": "argo_q_careteam_careteam_endpoint",
   "publisher": "The Argonaut Project",
   "status": "active",
   "target": [
      "Endpoint"
   ],
   "type": "reference",
   "url": "canon_base-for-argo-q/argo-q-careteam-careteam-endpoint",
   "resourceType": "SearchParameter"
}
1 ['Organization']
{
   "id": "argo-q-careteam-careteam-endpoin

### Create Markdown Text for Search.

- Using Jinja2 Template create markdown file for search section in profiles profiles

In [419]:
r_type = "CareTeam"  # TODO  loop over all types in sp_list

template = env.get_template(md_template[0])
sp_bytype = [i for i in sp_list if i.base[0] == r_type]
display(Markdown(template.render(sp_bytype=sp_bytype,search_type=search_type)))
search_md = template.render(sp_bytype=sp_bytype,search_type=search_type)

with open(f'{out_path}{r_type}-search.md','w') as f:
    f.write(search_md)



### Quick Start
Below is an overview of the required set of RESTful FHIR interactions - for
example, search and read operations - for this profile. See the [Conformance requirements]
for a complete list of supported RESTful interactions for this IG.

#### Supported Searches
1. **MAY** support search by the **`careteam-category`** parameter:
  `GET [base]/CareTeam?careteam-category={[system]}|[code]`
    
    - with support for these [search modifiers]: `text`
    
  *Implementation Notes:* Select CareTeams of the specified type. ([how to search by token]).
  <br />
  
1. **SHOULD** support search by the **`careteam-endpoint`** parameter:
  `GET [base]/CareTeam?careteam-endpoint=[url]`
    
    - for reference target types: `Endpoint`
    - with support for these [chained search parameters]:`careteam-endpoint:Endpoint.identifier`, `careteam-endpoint:Endpoint.connection-type`, `careteam-endpoint:Endpoint.organization`
    
  *Implementation Notes:* Select CareTeams with the specified endpoint. ([how to search by reference]).
  <br />
  
1. **SHALL** support search by the **`careteam-identifier`** parameter:
  `GET [base]/CareTeam?careteam-identifier={[system]}|[code]`
    
    - with support for these [search modifiers]: `text`, `ofType`
    
  *Implementation Notes:* Select CareTeams with the specified identifier. ([how to search by token]).
  <br />
  
1. **MAY** support search by the **`careteam-identifier-assigner`** parameter:
  `GET [base]/CareTeam?careteam-identifier-assigner=[url]`
    
    - for reference target types: `Organization`
    - with support for these [chained search parameters]:`careteam-identifier-assigner:Organization.identifier`, `careteam-identifier-assigner:Organization.name`
    
    - with support for these [search modifiers]: `below`
    
  *Implementation Notes:* Select care teams with an identifier assigned by the specified organization. ([how to search by reference]).
  <br />
  
1. **SHALL** support search by the **`careteam-location`** parameter:
  `GET [base]/CareTeam?careteam-location=[url]`
    
    - for reference target types: `Location`
    - with support for these [chained search parameters]:`careteam-location:Location.identifier`, `careteam-location:Location.type`, `careteam-location:Location.address`, `careteam-location:Location.organization`
    
    - with support for these [search modifiers]: `above`, `below`
    
  *Implementation Notes:* Select care teams operating at the specified location. ([how to search by reference]).
  <br />
  
1. **SHALL** support search by the **`careteam-member`** parameter:
  `GET [base]/CareTeam?careteam-member=[url]`
    
    - for reference target types: `PractitionerRole`, `Organization`, `CareTeam`
    - with support for these [search modifiers]: `type`
    
  *Implementation Notes:* Select care teams that include the specified member. ([how to search by reference]).
  <br />
  
1. **SHOULD** support search by the **`careteam-name`** parameter:
  `GET [base]/CareTeam?careteam-name=[string]`
    
    - with support for these [search modifiers]: `exact`, `contains`
    
  *Implementation Notes:* Select care teams with the specified name. ([how to search by string]).
  <br />
  
1. **SHALL** support search by the **`careteam-organization`** parameter:
  `GET [base]/CareTeam?careteam-organization=[url]`
    
    - for reference target types: `Organization`
    - with support for these [chained search parameters]:`careteam-organization:Organization.identifier`, `careteam-organization:Organization.name`, `careteam-organization:Organization.address`, `careteam-organization:Organization.partof`, `careteam-organization:Organization.type`
    
    - with support for these [search modifiers]: `above`, `below`
    
  *Implementation Notes:* Select care teams managed by the specified organization. ([how to search by reference]).
  <br />
  
1. **SHOULD** support search by the **`careteam-service`** parameter:
  `GET [base]/CareTeam?careteam-service=[url]`
    
    - for reference target types: `HealthcareService`
    - with support for these [chained search parameters]:`careteam-service:HealthcareService.identifier`, `careteam-service:HealthcareService.category`, `careteam-service:HealthcareService.organization`, `careteam-service:HealthcareService.location`
    
  *Implementation Notes:* Select care teams providing the specified service. ([how to search by reference]).
  <br />
  
1. **SHALL** support search by the **`careteam-status`** parameter:
  `GET [base]/CareTeam?careteam-status={[system]}|[code]`
    
  *Implementation Notes:* Select CareTeams with the specified status. ([how to search by token]).
  <br />
  
1. **MAY** support search by the **`careteam-via-intermediary`** parameter:
  `GET [base]/CareTeam?careteam-via-intermediary=[url]`
    
    - for reference target types: `PractitionerRole`, `Organization`, `OrganizationAffiliation`, `Location`
  *Implementation Notes:* Select care teams with contact information available through the specified intermediary. ([how to search by reference]).
  <br />
  

{% include link-list.md %}

### Create Markdown Text for SearchParameters Page

- Using Jinja2 Template create markdown file for searchparameters page

In [420]:
# get types:
r_list = []
for i in sp_list:
    for j in i.base:
        r_list.append(j)

r_list = sorted(set(r_list))

# md_template = 'sp_list_page.j2'

template = env.get_template(md_template[1])
display(Markdown(template.render(sp_list=sp_list,r_list=r_list)))
searchparameters_md = template.render(sp_list=sp_list,r_list=r_list)
with open(f'{out_path}searchparameters.md','w') as f:
    f.write(searchparameters_md)

---
title: Operations and Search Parameters
layout: default
topofpage: true
sectionnumbering: true
---

The following search parameters have been defined for the {{site.data.fhir.igName}} Implementation Guide.  For more information on the [FHIR RESTful search api]and the standard [Search Param Registry] see the FHIR specification.

<!-- Operations


  { % include list-simple-operationdefinitions.xhtml % }


-->

Search Parameter


**CareTeam**
  - [careteam-category](SearchParameter-argo-q-careteam-careteam-category.html)
  - [careteam-endpoint](SearchParameter-argo-q-careteam-careteam-endpoint.html)
  - [careteam-identifier](SearchParameter-argo-q-careteam-careteam-identifier.html)
  - [careteam-identifier-assigner](SearchParameter-argo-q-careteam-careteam-identifier-assigner.html)
  - [careteam-location](SearchParameter-argo-q-careteam-careteam-location.html)
  - [careteam-member](SearchParameter-argo-q-careteam-careteam-member.html)
  - [careteam-name](SearchParameter-argo-q-careteam-careteam-name.html)
  - [careteam-organization](SearchParameter-argo-q-careteam-careteam-organization.html)
  - [careteam-service](SearchParameter-argo-q-careteam-careteam-service.html)
  - [careteam-status](SearchParameter-argo-q-careteam-careteam-status.html)
  - [careteam-via-intermediary](SearchParameter-argo-q-careteam-careteam-via-intermediary.html)

**Endpoint**
  - [endpoint-connection-type](SearchParameter-argo-q-endpoint-endpoint-connection-type.html)
  - [endpoint-identifier](SearchParameter-argo-q-endpoint-endpoint-identifier.html)
  - [endpoint-identifier-assigner](SearchParameter-argo-q-endpoint-endpoint-identifier-assigner.html)
  - [endpoint-mime-type](SearchParameter-argo-q-endpoint-endpoint-mime-type.html)
  - [endpoint-organization](SearchParameter-argo-q-endpoint-endpoint-organization.html)
  - [endpoint-payload-type](SearchParameter-argo-q-endpoint-endpoint-payload-type.html)
  - [endpoint-status](SearchParameter-argo-q-endpoint-endpoint-status.html)
  - [endpoint-usecase-standard](SearchParameter-argo-q-endpoint-endpoint-usecase-standard.html)
  - [endpoint-usecase-type](SearchParameter-argo-q-endpoint-endpoint-usecase-type.html)
  - [endpoint-via-intermediary](SearchParameter-argo-q-endpoint-endpoint-via-intermediary.html)

**HealthcareService**
  - [active](SearchParameter-argo-q-healthcareservice-active.html)
  - [healthcareservice-category](SearchParameter-argo-q-healthcareservice-healthcareservice-category.html)

**Organization**
  - [active](SearchParameter-argo-q-organization-active.html)
  - [address-use](SearchParameter-argo-q-organization-address-use.html)

**OrganizationAffiliation**
  - [active](SearchParameter-argo-q-organizationaffiliation-active.html)

**Practitioner**
  - [active](SearchParameter-argo-q-practitioner-active.html)

**PractitionerRole**
  - [active](SearchParameter-argo-q-practitionerrole-active.html)

{% include link_list.md %}

### Update CapabilityStatement for Search
- instantiate new or existing CapStatement as fhir model
- add combination extensions
- add markdown descriptions from above

In [421]:
# instantiate new or existing CapStatement:

def interaction(code):
    return CS.CapabilityStatementRestResourceInteraction({'code' : code})

def search_cs(sp):
    template = env.get_template(md_template[2])
    return CS.CapabilityStatementRestResourceSearchParam(
    dict(
    name = sp.code,
    definition = sp.url,
    type = sp.type,
    documentation = template.render(sp=sp, search_type=search_type)
    )
    )

cs = CS.CapabilityStatement(dict(
                            date = now,
                            status = 'active',
                            acceptUnknown = 'both',
                            format = ['json','xml'],
                            fhirVersion = '3.0.1',
                            kind = 'requirements'     
                            ))



cs.rest = [CS.CapabilityStatementRest()]  
cs.rest[0].mode = 'server'
cs.rest[0].resource = []



print(r_list)    
for r in r_list:
    rest_resource = CS.CapabilityStatementRestResource()
    rest_resource.type = r
    rest_resource.interaction = [interaction(code) for code in ['read','vread', 'history-instance', 'search-type']]
    cs.rest[0].resource.append(rest_resource)
    rest_resource.searchParam = [search_cs(sp=sp) for sp in sp_list if sp.base[0] == r]
print(dumps(cs.as_json(),indent = 3))
    
    



['CareTeam', 'Endpoint', 'HealthcareService', 'Organization', 'OrganizationAffiliation', 'Practitioner', 'PractitionerRole']
{
   "acceptUnknown": "both",
   "date": "2019-01-28T19:48:32.472989",
   "fhirVersion": "3.0.1",
   "format": [
      "json",
      "xml"
   ],
   "kind": "requirements",
   "rest": [
      {
         "mode": "server",
         "resource": [
            {
               "interaction": [
                  {
                     "code": "read"
                  },
                  {
                     "code": "vread"
                  },
                  {
                     "code": "history-instance"
                  },
                  {
                     "code": "search-type"
                  }
               ],
               "searchParam": [
                  {
                     "definition": "canon_base-for-argo-q/argo-q-careteam-careteam-category",
                     "documentation": "\n**MAY** support search by the **`careteam-category`** 

### TODO

- Save SearchParameters and CapabilityStatements.
- Cycle through the quick starts
- Update the CSV files